# 02: Reduce Bonn PT stops to Tier operating area

*Note: This notebook does not need to be re-run for every single month*

Note: Originally run on local machine

In the last notebook, we reduced the PT stops to a "Bonn buffer zone". But there are still stations, that either do no belong to Bonn or lie outside of the Tier operating are. Because of this, we now check, at which stations a Tier trip happened. Only stations whith a Tier trip stay within the data.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [ ]:
month_single = '7'
month_double = '07'

In [2]:
# load Bonn stops
stops = gpd.read_parquet('../Data/01_bonn_stop_buffer.parquet')

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2065/2549450356.py:2: UserWarning: Multiple non-primary geometry columns read from Parquet/Feather file. The first column read was promoted to the primary geometry.
  stops = gpd.read_parquet('/Volumes/T7/Masterarbeit/stop_buffer.parquet')


In [3]:
# create a GeoDataFrame
stops = gpd.GeoDataFrame(stops)

In [4]:
# load Tier data
tier = gpd.read_parquet("../Data/11_tier_part1")

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2065/2223975070.py:2: UserWarning: Multiple non-primary geometry columns read from Parquet/Feather file. The first column read was promoted to the primary geometry.
  tier = gpd.read_parquet("/Volumes/T7/Masterarbeit/tier.parquet")


In [5]:
# reduce data to one month for easier handling
tier_one_month = tier[tier['tier_trips_start_time'].dt.month == 7]


In [6]:
#change lat and long
tier_one_month['start_location'] = tier_one_month['start_location'].apply(lambda point: Point(point.y, point.x))
tier_one_month['end_location'] = tier_one_month['end_location'].apply(lambda point: Point(point.y, point.x))

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
stops.reset_index(drop = True, inplace=True)
tier_one_month.reset_index(drop = True, inplace=True)

In [8]:

# Create an empty dataframe to store the matching rows
matching_rows = pd.DataFrame(columns=stops.columns)

# Iterate through test_stop
for index, stop_row in stops.iterrows():
    # Check if any buffer_zone in test_stops contains the point in test_tier start_location
    contains_point = any(stop_row['buffer_zone'].contains(point) for point in tier_one_month['start_location'])
    
    if contains_point:
        # Append the row to the matching_rows dataframe
        matching_rows = pd.concat([matching_rows, stop_row])
    else:
        # Delete the row from the test_stop dataframe
        stops.drop(index, inplace=True)

# Reset the index of the test_stop dataframe
stops.reset_index(drop=True, inplace=True)


In [9]:
stops

,station_point,buffer_zone,lat,long
0,POINT (7.07351 50.73910),"POLYGON ((7.07451 50.73910, 7.07450 50.73900, ...",50.739096,7.073507
1,POINT (7.15955 50.68392),"POLYGON ((7.16055 50.68392, 7.16055 50.68382, ...",50.683920,7.159553
2,POINT (7.15835 50.68034),"POLYGON ((7.15935 50.68034, 7.15934 50.68024, ...",50.680336,7.158347
3,POINT (7.04275 50.77002),"POLYGON ((7.04375 50.77002, 7.04374 50.76992, ...",50.770021,7.042745
4,POINT (7.01750 50.74327),"POLYGON ((7.01850 50.74327, 7.01849 50.74317, ...",50.743266,7.017498
...,...,...,...,...
488,POINT (7.01584 50.75370),"POLYGON ((7.01684 50.75370, 7.01684 50.75360, ...",50.753700,7.015840
489,POINT (7.04793 50.74116),"POLYGON ((7.04893 50.74116, 7.04892 50.74106, ...",50.741163,7.047926
490,POINT (7.07265 50.72710),"POLYGON ((7.07365 50.72710, 7.07365 50.72700, ...",50.727103,7.072652
491,POINT (7.07273 50.73711),"POLYGON ((7.07373 50.73711, 7.07372 50.73701, ...",50.737108,7.072725


In [10]:
# export redcued stops as parquet
stops.to_parquet('..//all_bonn_stop_buffers.parquet') 

In [11]:

# Create a map centered at a specific location
m = folium.Map(location=[50.756861, 7.002714], zoom_start=12)

# Add the buffer zones as polygons to the map
for index, row in stops.iterrows():
    folium.GeoJson(row['buffer_zone']).add_to(m)

# Display the map
m